In [ ]:
#Data Extraction using Beautiful Soup
def data_extraction():
    #Libraries For Data Extraction
    from bs4 import BeautifulSoup
    import time
    import datetime
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By
    from selenium import webdriver
    
    #specify the url
    url = 'http://mmdatraffic.interaksyon.com/line-view-edsa.php'
    
    # Start the WebDriver and load the page
    wd = webdriver.Firefox()
    wd.get(url)

    # Wait for the dynamically loaded elements to show up
    WebDriverWait(wd, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "line-col")))

    # And grab the page HTML source
    html_page = wd.page_source
    wd.quit()

    # Now you can use html_page as you like
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html_page)

    #set time stamp.
    ts = time.time()
    timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H_%M_%S')
    stringdata = "TIMESTAMP: "+ str(timestamp)+"\n"+"\n"

    #extract name of lines
    list_of_names_html=soup.find_all('div',{'class':'line-name'})
    list_of_names = []

    for children in list_of_names_html:
        grandchildren = children.findChild("p")
        temp = grandchildren.get_text(separator=' ')
        temp = temp.split(' ')
        list_of_names.append(temp[0]) #NOTE: fix string separation
    list_of_names.pop(0) 

    #extract southbound/northbound volume
    list_of_volume_html=soup.find_all('div',{'class':'line-status'})
    list_of_southbound = []
    list_of_northbound = []

    i=1
    for children in list_of_volume_html:
        temp = children.text
        temp = temp.split()

        if(temp[1]=="LIGHT"):
            temp[1]=0
        elif(temp[1]=="MODERATE"):
            temp[1]=1
        elif(temp[1]=="HEAVY"):
            temp[1]=2

        if(i%2 == 0):
            list_of_northbound.append(temp[1])
        else:
            list_of_southbound.append(temp[1])

        i=i+1

    #save data into one variable
    for i in range(len(list_of_names)):
        stringdata = stringdata + '{:>12}  {:>12}  {:>12}'.format(list_of_names[i], str(list_of_southbound[i]), str(list_of_northbound[i])) + "\n"
    
    #Save raw data into csv file
    filename = "rawdata_"+ str(timestamp)+".csv"
    file = open(filename,"a")
    file.write(stringdata)   
    file.close()
    
    #return list containing data lists
    list_of_lists = [list_of_names, list_of_southbound, list_of_northbound, timestamp]
    return list_of_lists